# 🤖 TARS v3 — Google Colab Full Training Pipeline

**Полное обучение ТАРС v3: мозг + голос (STT + TTS)**

### Подход: Квантизация → Дообучение (QAT)
Сначала модель квантуется в 1.58-bit, затем дообучается в квантованном режиме через STE.
Это даёт лучшее качество, чем квантовать после обучения.

### Фазы обучения:
| # | Модуль | Подход | Время (T4) |
|---|--------|--------|------------|
| 2 | Рефлексы | MinGRU Classifier | ~2 мин |
| 3 | MinGRU LM | System 1 генератор | ~30 мин |
| 4.1 | Mamba-2 Warmup | FP16 pretrain (3 эпохи) | ~1 ч |
| 4.Q | **Квантизация** | FP16 → 1.58-bit BitNet | ~1 мин |
| 4.2 | Mamba-2 Phase 2 | **1.58-bit STE** (WKV+Fusion) | ~1.5 ч |
| 4.3 | Mamba-2 Phase 3 | **1.58-bit STE** (MoLE+Pool) | ~1 ч |
| 4.4 | Mamba-2 Phase 4 | **1.58-bit STE** (RAG+Memory) | ~1 ч |
| 7 | Валидация | Тестовая генерация | ~5 мин |
| 8 | Whisper STT | LoRA fine-tune (base, русский) | ~40 мин |
| 9 | Piper TTS | VITS fine-tune (русский голос) | ~1-2 ч |
| 10 | Voice INT8 | Квантизация ONNX + Whisper Boost | ~5 мин |

### Инструкции:
1. **Runtime → Change runtime type → GPU (T4)**
2. Запускай ячейки по порядку ▶
3. Модели автоматически сохраняются на **Google Drive**
4. При отключении Colab — запусти ячейку 0, модели восстановятся с Drive

---

## 0. Проверка GPU + Google Drive + Восстановление моделей

In [ ]:
import torch
import os
import time
import shutil

print("=" * 65)
print("  TARS v3 — Quant-First Training Pipeline")
print("=" * 65)

if torch.cuda.is_available():
    GPU_NAME = torch.cuda.get_device_name(0)
    VRAM_GB = torch.cuda.get_device_properties(0).total_mem / 1024**3
    print(f"  ✅ GPU:  {GPU_NAME}")
    print(f"  ✅ VRAM: {VRAM_GB:.1f} GB")
    print(f"  ✅ CUDA: {torch.version.cuda}")
else:
    print("  ❌ GPU НЕ ОБНАРУЖЕН!")
    raise RuntimeError("GPU not available")

if VRAM_GB >= 35:
    BATCH, ACCUM = 32, 2
    WHISPER_BATCH, WHISPER_MODEL = 32, "base"
    PIPER_BATCH = 32
    GPU_TIER = "A100"
elif VRAM_GB >= 14:
    BATCH, ACCUM = 16, 4
    WHISPER_BATCH, WHISPER_MODEL = 16, "base"
    PIPER_BATCH = 16
    GPU_TIER = "T4"
else:
    BATCH, ACCUM = 8, 8
    WHISPER_BATCH, WHISPER_MODEL = 8, "tiny"
    PIPER_BATCH = 8
    GPU_TIER = "Budget"

print(f"  ⚡ Tier: {GPU_TIER} (batch={BATCH}×{ACCUM}={BATCH*ACCUM})")
print(f"  🎤 Whisper: {WHISPER_MODEL}")

SAVE_TO_DRIVE = True
DRIVE_DIR = "/content/drive/MyDrive/TARS_v3_models"

if SAVE_TO_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    os.makedirs(DRIVE_DIR, exist_ok=True)
    print(f"  💾 Google Drive: {DRIVE_DIR}")

print("=" * 65)
T_START = time.time()

## 1. Клонирование + Восстановление с Drive

In [ ]:
import os, sys, shutil, glob

REPO_URL = "https://github.com/Vazilll/TarsSSM-Py.git"
WORK_DIR = "/content/TarsSSM-Py"

if os.path.exists(os.path.join(WORK_DIR, "mega_train.py")):
    print("✅ Репозиторий уже клонирован")
    !cd {WORK_DIR} && git pull --ff-only 2>/dev/null || true
else:
    !git clone {REPO_URL} {WORK_DIR}

os.chdir(WORK_DIR)
sys.path.insert(0, WORK_DIR)

!pip install -q einops tqdm psutil sentencepiece tokenizers \
    sentence-transformers datasets transformers peft jiwer \
    faster-whisper onnxruntime sounddevice

# ═══ ВОССТАНОВЛЕНИЕ МОДЕЛЕЙ С DRIVE ═══
if SAVE_TO_DRIVE and os.path.exists(DRIVE_DIR):
    restore_map = {
        "mamba2_final.pt":   "models/mamba2/mamba2_omega.pt",
        "mamba2_phase3.pt": "models/mamba2/mamba2_omega.pt",
        "mamba2_phase2.pt": "models/mamba2/mamba2_omega.pt",
        "mamba2_phase1.pt": "models/mamba2/mamba2_omega.pt",
        "mamba2_158bit.pt": "models/mamba2/mamba2_omega_158bit.pt",
        "mamba2_fp16.pt":   "models/mamba2/mamba2_omega.pt",
        "mingru.pt":        "models/mingru_weights.pt",
        "reflex.pt":        "models/reflex/reflex_classifier.pt",
    }
    restored = 0
    for drive_name, local_path in restore_map.items():
        drive_path = os.path.join(DRIVE_DIR, drive_name)
        local_full = os.path.join(WORK_DIR, local_path)
        if os.path.exists(drive_path) and not os.path.exists(local_full):
            os.makedirs(os.path.dirname(local_full), exist_ok=True)
            shutil.copy2(drive_path, local_full)
            size_mb = os.path.getsize(local_full) / 1024 / 1024
            print(f"  🔄 Drive → {local_path} ({size_mb:.1f} MB)")
            restored += 1
    drive_whisper = os.path.join(DRIVE_DIR, "whisper_ru_lora")
    local_whisper = os.path.join(WORK_DIR, "models", "voice", "whisper_ru_lora")
    if os.path.exists(drive_whisper) and not os.path.exists(local_whisper):
        shutil.copytree(drive_whisper, local_whisper)
        print(f"  🔄 Drive → whisper_ru_lora/")
        restored += 1
    if restored > 0:
        print(f"  ✅ Восстановлено {restored} моделей с Drive")
    else:
        print("  ℹ На Drive нет моделей — обучение с нуля")

def save_to_drive(src_path, name=None):
    if not SAVE_TO_DRIVE or not os.path.exists(src_path):
        return
    dst = os.path.join(DRIVE_DIR, name or os.path.basename(src_path))
    shutil.copy2(src_path, dst)
    size_mb = os.path.getsize(dst) / 1024 / 1024
    print(f"  💾 → Drive: {name or os.path.basename(src_path)} ({size_mb:.1f} MB)")

print("✅ Ready")

## 2. Скачивание данных (Wiki + HF + Identity)

In [ ]:
import glob
os.makedirs("data", exist_ok=True)

# Wikipedia
wiki_path = os.path.join(WORK_DIR, "data", "wiki_ru.txt")
if os.path.exists(wiki_path) and os.path.getsize(wiki_path) > 100_000:
    print(f"📚 Wikipedia: уже есть ({os.path.getsize(wiki_path)/1024/1024:.1f} MB)")
else:
    print("📚 Скачивание Wikipedia (50 000 статей)...")
    !python training/download_wiki.py --count 50000

# HuggingFace
hf_files = glob.glob(os.path.join(WORK_DIR, "data", "hf_*.txt"))
if len(hf_files) >= 1:
    total_mb = sum(os.path.getsize(f) for f in hf_files) / 1024 / 1024
    print(f"🤗 HuggingFace: уже есть ({len(hf_files)} файлов, {total_mb:.0f} MB)")
else:
    !python training/download_hf_dataset.py --preset all

# TARS Identity (self-description)
identity_path = os.path.join(WORK_DIR, "data", "tars_identity.txt")
if os.path.exists(identity_path):
    size_kb = os.path.getsize(identity_path) / 1024
    print(f"🧬 ТАРС Identity: {size_kb:.0f} KB (самоописание архитектуры)")
else:
    print("⚠ tars_identity.txt не найден!")

# LEANN
try:
    emb_path = os.path.join(WORK_DIR, "models", "embeddings")
    if not os.path.exists(emb_path):
        from sentence_transformers import SentenceTransformer
        SentenceTransformer('all-MiniLM-L6-v2').save(emb_path)
    print(f"🧠 Embeddings: OK")
except Exception as e:
    print(f"  ⚠ Embeddings: {e}")

try:
    sys.path.insert(0, os.path.join(WORK_DIR, "training"))
    from ingest_to_leann import ingest_all
    ingest_all()
    print("🧠 LEANN: OK")
except Exception as e:
    print(f"  ℹ LEANN: {e}")

data_files = [f for f in glob.glob(os.path.join(WORK_DIR, "data", "*")) if os.path.isfile(f)]
print(f"\n📊 Итого: {sum(os.path.getsize(f) for f in data_files)/1024/1024:.0f} MB")

---
# 🧠 МОЗГ
---

## Фаза 2: Рефлексы (~2 мин)

In [ ]:
print("="*65)
print("  Фаза 2: Рефлексы (MinGRU Classifier)")
print("="*65)
!python training/train_reflex.py --epochs 100 --lr 0.002
save_to_drive("models/reflex/reflex_classifier.pt", "reflex.pt")

## Фаза 3: MinGRU LM — System 1 (~30 мин)

In [ ]:
print("="*65)
print("  Фаза 3: MinGRU Language Model")
print("="*65)
!python training/train_mingru.py \
    --epochs 50 --lr 3e-3 --dim 512 --layers 6 \
    --batch 32 --seq_len 256 --augment
save_to_drive("models/mingru_weights.pt", "mingru.pt")

## Фаза 4.1: FP16 Warmup (3 эпохи)

Короткий прогрев в FP16 — модель учит базовые паттерны и **свою архитектуру** из `tars_identity.txt`.

~1 час на T4

In [ ]:
import torch, os

device = "cuda"
print(f"GPU: {torch.cuda.get_device_name(0)} ({VRAM_GB:.0f} GB)")
print(f"Batch: {BATCH}×{ACCUM} = {BATCH*ACCUM} effective")

# Transfer embedding MinGRU → Mamba-2
emb_args = ""
mingru_path = os.path.join(WORK_DIR, "models", "mingru_weights.pt")
if os.path.exists(mingru_path):
    cp = torch.load(mingru_path, map_location='cpu', weights_only=False)
    state = cp.get('model_state_dict', cp)
    for k in state:
        if 'shared_embedding' in k or 'emb.weight' in k:
            emb_dir = os.path.join(WORK_DIR, "models", "tars_v3")
            os.makedirs(emb_dir, exist_ok=True)
            emb_path = os.path.join(emb_dir, "_transfer_embedding.pt")
            torch.save(state[k], emb_path)
            emb_args = f"--pretrained_emb {emb_path}"
            print(f"🔗 Transfer embedding: {state[k].shape}")
            break

BASE = f"--d_model 768 --n_layers 12 --vocab_size 256 --batch {BATCH} --accum_steps {ACCUM} --device {device} --curriculum --label_smoothing 0.1 {emb_args}"

In [ ]:
# Phase 1: FP16 Warmup — 3 эпохи (все компоненты)
print("="*65)
print("  Phase 4.1: FP16 Warmup (3 эпохи, все компоненты)")
print("="*65)
!python training/train_mamba2.py {BASE} --epochs 3 --lr 3e-4 --phase 1 --seq_len 256
save_to_drive("models/mamba2/mamba2_omega.pt", "mamba2_fp16.pt")

## Фаза 4.Q: Квантизация FP16 → 1.58-bit

**Ключевой момент:** квантуем СНАЧАЛА, потом дообучаем.

Модель конвертируется в 1.58-bit (BitNet), затем все следующие фазы идут в квантованном режиме через STE.

~1 мин

In [ ]:
# Квантизация: FP16 → 1.58-bit + STE дообучение (5 эпох)
print("="*65)
print("  Phase 4.Q: Квантизация FP16 → 1.58-bit BitNet + STE warmup")
print("="*65)

fp16_path = os.path.join(WORK_DIR, "models", "mamba2", "mamba2_omega.pt")
if os.path.exists(fp16_path):
    fp16_mb = os.path.getsize(fp16_path) / 1024 / 1024
    print(f"  FP16 модель: {fp16_mb:.0f} MB")
    # Фаза квантизации: загружает FP16, конвертирует в 1.58-bit, дообучает 5 эпох
    !python training/train_mamba2.py {BASE} \
        --epochs 5 --lr 5e-5 --phase 1 --quant \
        --resume --seq_len 256
    save_to_drive("models/mamba2/mamba2_omega_158bit.pt", "mamba2_158bit.pt")
else:
    print("⚠ FP16 модель не найдена")

## Фаза 4.2: WKV + Fusion in 1.58-bit (5 эпох)

Дообучение **уже квантованной** модели. SSD заморожен, тренируются WKV + Fusion + Ω-SSM + NoveltyGate.

~1.5 часа на T4

In [ ]:
print("="*65)
print("  Phase 4.2: WKV + Fusion (1.58-bit STE)")
print("="*65)
!python training/train_mamba2.py {BASE} \
    --epochs 5 --lr 1e-4 --phase 2 --quant \
    --resume --seq_len 512
save_to_drive("models/mamba2/mamba2_omega_158bit.pt", "mamba2_158bit_p2.pt")

## Фаза 4.3: MoLE + MatrixPool in 1.58-bit (3 эпохи)

~1 час на T4

In [ ]:
print("="*65)
print("  Phase 4.3: MoLE + MatrixPool (1.58-bit STE)")
print("="*65)
!python training/train_mamba2.py {BASE} \
    --epochs 3 --lr 3e-5 --phase 3 --quant \
    --resume --seq_len 512
save_to_drive("models/mamba2/mamba2_omega_158bit.pt", "mamba2_158bit_p3.pt")

## Фаза 4.4: RAG + Memory in 1.58-bit (3 эпохи)

~1 час на T4

In [ ]:
print("="*65)
print("  Phase 4.4: RAG + Memory + Spine (1.58-bit STE)")
print("="*65)
!python training/train_mamba2.py {BASE} \
    --epochs 3 --lr 1.5e-5 --phase 4 --quant \
    --resume --seq_len 512
save_to_drive("models/mamba2/mamba2_omega_158bit.pt", "mamba2_158bit.pt")

## Фаза 7: Валидация мозга

In [ ]:
import torch, sys, gc
gc.collect()
torch.cuda.empty_cache()
sys.path.insert(0, WORK_DIR)

print("="*65)
print("  Фаза 7: Валидация мозга")
print("="*65)

try:
    from brain.tokenizer import TarsTokenizer
    from brain.mamba2.model import TarsMamba2LM

    tokenizer = TarsTokenizer()
    model, ckpt = TarsMamba2LM.load_pretrained(device="cuda")
    model.eval()

    if ckpt:
        params = sum(p.numel() for p in model.parameters())
        print(f"✅ Model: {ckpt}")
        print(f"✅ Params: {params:,}\n")

        for prompt in ["привет", "кто ты", "что такое", "из чего ты состоишь", "помоги"]:
            tokens = tokenizer.encode(prompt)
            input_ids = torch.tensor([tokens], dtype=torch.long, device="cuda")
            with torch.no_grad():
                logits = model(input_ids)
            probs = torch.softmax(logits[0, -1, :], dim=-1)
            top5 = torch.topk(probs, 5)
            preds = [f"'{tokenizer.decode([i])}'({p:.1%})" for i, p in zip(top5.indices.tolist(), top5.values.tolist())]
            print(f"  '{prompt}' → {', '.join(preds)}")
        print("\n✅ Brain works!")
    else:
        print("⚠ No weights")
    del model
    torch.cuda.empty_cache()
except Exception as e:
    print(f"❌ {e}")

---
# 🎤 ГОЛОС
---

## Фаза 8: Whisper STT — base (74M, LoRA, ~40 мин)

In [ ]:
print("="*65)
print(f"  Фаза 8: Whisper {WHISPER_MODEL} LoRA Fine-tune (Russian STT)")
print("="*65)

import gc, torch
gc.collect()
torch.cuda.empty_cache()

!python training/train_whisper.py \
    --model {WHISPER_MODEL} --device cuda \
    --samples 10000 --val_samples 1000 \
    --epochs 5 --batch {WHISPER_BATCH} \
    --lr 1e-3 --lora_r 32

whisper_dir = os.path.join(WORK_DIR, "models", "voice", "whisper_ru_lora")
if os.path.exists(whisper_dir) and SAVE_TO_DRIVE:
    drive_whisper = os.path.join(DRIVE_DIR, "whisper_ru_lora")
    if os.path.exists(drive_whisper):
        shutil.rmtree(drive_whisper)
    shutil.copytree(whisper_dir, drive_whisper)
    print(f"  💾 → Drive: whisper_ru_lora/")
    print("✅ Whisper STT обучен!")

## Фаза 9: Piper TTS (~1-2 ч)

In [ ]:
print("="*65)
print("  Фаза 9: Piper TTS Fine-tune (Russian Voice)")
print("="*65)

gc.collect()
torch.cuda.empty_cache()
!pip install -q piper-tts piper-phonemize 2>/dev/null || true

!python training/train_piper.py --epochs 1000 --max_samples 3000 --batch {PIPER_BATCH}

piper_onnx = os.path.join(WORK_DIR, "models", "voice", "tars_voice_ru.onnx")
if os.path.exists(piper_onnx):
    save_to_drive(piper_onnx, "tars_voice_ru.onnx")
    print("✅ Piper TTS обучен!")

## Фаза 10: Whisper Boost + Voice INT8 (~5 мин)

In [ ]:
print("="*65)
print("  Фаза 10: Whisper Boost + Voice INT8")
print("="*65)

!python training/whisper_boost.py
whisper_ctx = os.path.join(WORK_DIR, "models", "voice", "whisper_context.json")
if os.path.exists(whisper_ctx):
    save_to_drive(whisper_ctx, "whisper_context.json")

!python training/quantize_voice.py
print("\n✅ Voice pipeline done!")

---
# 📦 Финальная сборка
---

In [ ]:
import shutil, json, time

OUTPUT_DIR = "/content/tars_v3_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*65)
print("  Финальная сборка")
print("="*65)

model_files = {
    "mamba2_158bit.pt": os.path.join(WORK_DIR, "models", "mamba2", "mamba2_omega_158bit.pt"),
    "mamba2_fp16.pt":   os.path.join(WORK_DIR, "models", "mamba2", "mamba2_omega.pt"),
    "mingru.pt":        os.path.join(WORK_DIR, "models", "mingru_weights.pt"),
    "reflex.pt":        os.path.join(WORK_DIR, "models", "reflex", "reflex_classifier.pt"),
    "tars_voice_ru.onnx": os.path.join(WORK_DIR, "models", "voice", "tars_voice_ru.onnx"),
    "whisper_context.json": os.path.join(WORK_DIR, "models", "voice", "whisper_context.json"),
}

saved = []
for name, src in model_files.items():
    if os.path.exists(src):
        shutil.copy2(src, os.path.join(OUTPUT_DIR, name))
        mb = os.path.getsize(src) / 1024 / 1024
        print(f"  ✅ {name} ({mb:.1f} MB)")
        saved.append(name)
        save_to_drive(src, name)
    else:
        print(f"  ⏭ {name}")

# Whisper LoRA
wl_src = os.path.join(WORK_DIR, "models", "voice", "whisper_ru_lora")
wl_dst = os.path.join(OUTPUT_DIR, "whisper_ru_lora")
if os.path.exists(wl_src):
    if os.path.exists(wl_dst): shutil.rmtree(wl_dst)
    shutil.copytree(wl_src, wl_dst)
    print(f"  ✅ whisper_ru_lora/")
    saved.append("whisper_ru_lora/")

# Archive
arc = shutil.make_archive("/content/tars_v3_full", 'gztar', OUTPUT_DIR)
if SAVE_TO_DRIVE:
    shutil.copy2(arc, os.path.join(DRIVE_DIR, "tars_v3_full.tar.gz"))

total_time = time.time() - T_START
print(f"\n{'='*65}")
print(f"  ИТОГО: {total_time/3600:.1f}ч | {len(saved)} моделей")
for m in saved: print(f"     ✅ {m}")
print(f"  📁 {arc} ({os.path.getsize(arc)/1024/1024:.0f} MB)")
print(f"  💾 {DRIVE_DIR}")
print(f"{'='*65}")
print(f"  🎯 ТАРС v3 ОБУЧЕН (Quant-First)!")

In [ ]:
from google.colab import files
files.download("/content/tars_v3_full.tar.gz")